In [7]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.backends import default_backend


In [8]:
# Generated keys ECC
private_key = ec.generate_private_key(ec.SECP384R1(), default_backend())
public_key = private_key.public_key()

# Function for signature message
def sign_message(private_key, message):
    signature = private_key.sign(
        message,
        ec.ECDSA(hashes.SHA256())
    )
    return signature

# Function to verify the message signature
def verify_signature(public_key, message, signature):
    try:
        public_key.verify(
            signature,
            message,
            ec.ECDSA(hashes.SHA256())
        )
        return True
    except InvalidSignature:
        return False

# Function for encrypted message
def encrypt_message(key, message):
    cipher = Cipher(algorithms.AES(key), modes.CFB8(b'0000000000000000'), backend=default_backend())
    encryptor = cipher.encryptor()
    return encryptor.update(message) + encryptor.finalize()

# Function for decrypted message
def decrypt_message(key, ciphertext):
    cipher = Cipher(algorithms.AES(key), modes.CFB8(b'0000000000000000'), backend=default_backend())
    decryptor = cipher.decryptor()
    return decryptor.update(ciphertext) + decryptor.finalize()

# Message encrypred
original_message = b"secret message"

# Signature of message
signature = sign_message(private_key, original_message)
print("Signature of original message :", signature)

# Generated from secret shared from the recipient's private key and public key
shared_secret = private_key.exchange(ec.ECDH(), public_key)

# Using HKDF to derive a key from the shared secret
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_secret)

# Encrypted message
encrypted_message = encrypt_message(derived_key, original_message)
print(" encrypted Message:", encrypted_message)

# Decrypted message
decrypted_message = decrypt_message(derived_key, encrypted_message)
print(" decrypted message:", decrypted_message)

# Signature Verification
if not verify_signature(public_key, decrypted_message, signature):
    raise ValueError(" verification of the signature is failed .")

# Display
print("Processus of encrypted and decrypted is a succes.")
print(" original message:", original_message)
print(" decrypted message :", decrypted_message)


Signature of original message : b"0e\x020W\x9d\xd4kC\xa5}\xed`\x8eB\x1fBH<\x84\x15\x11\xacn\xb4HY\xfb\x98\x80u<*\xf4'%\x9eI\xb7\xb3\xa2\xc5#\x9e\xe6\x02\xb0\xaa\xe5\x1d$G\x021\x00\xae\x08\x8bs\x1a\xc0\xf4\xfdk\xc1\x82\xf7&\x85\x90\x86v\x16r&\xbd\xc9\xe2\x13\xe4\xbc\xf4\x8cM?.\xb8\nl\x00%\x1b\xdb\xb7hUL\xb0s\x93J\x12|"
 encrypted Message: b'\xb5f\xa8\x83\xff\\qn\xb1\xb9\x94oj\xa7'
 decrypted message: b'secret message'
Processus of encrypted and decrypted is a succes.
 original message: b'secret message'
 decrypted message : b'secret message'
